### Algorithm 1: Locate-Termination

In [6]:
from scipy.stats import poisson
import pysam
import pandas as pd
import numpy as np

In [ ]:
# Poisson inverse cumulative distribution function
poisson.ppf(0.01, 20)

In [ ]:
# Load the BED file
gene_names = []
gene_loc = []
for line in open( "mm10_final.bed" ):
    fields = line.split( "\t" )
    chrom = fields[0]
    loc1 = fields[1]
    loc2 = fields[2]
    strand = fields[4][1:-1]
    loc = [chrom, loc1, loc2, strand]
    gene_loc.append(loc)
    gene_names.append(fields[3])

samfile = pysam.AlignmentFile("end_P2_E01_529_mono_01.sorted.bam", "rb")

In [ ]:
# for read in samfile.fetch('1', 30000000, 70000000):
#      print(read)

# samfile.count('1', start=30000000, stop=70000000)
hed = samfile.head(1)
# print(hed.next_reference_start())
for h in hed:
    print(h.get_reference_positions())
# Read count of a transcript (1st)
first_loc = 0
hed = samfile.head(1)
for h in hed:
    first_loc = h.get_reference_positions()[0]
first_loc

In [ ]:
# Example of algorithm over one transcript (calculating window counts)
first_loc = 4845673
# first_loc = 5149991
chrom = '1'
l_g = 1000
N = samfile.count(chrom, start=first_loc, end=first_loc+l_g)
windows = []
threshold = 0.01
s_t = 75
s_w = 100
l = 0
n_w = (l_g - s_w)/s_t + 1
N_w = N * s_w / l_g 
m = poisson.ppf(threshold, N_w)
start_wi = first_loc
for i in range(int(n_w)):
    count = samfile.count(chrom, start=start_wi, end=start_wi+s_w)
    windows.append(count)
    start_wi += s_t
windows

In [ ]:
# Example of algorithm output over one transcript (with introns)
first_loc = 5149991
chrom = '1'
strand = '+'
N = samfile.count(chrom, start=first_loc, end=first_loc+l_g)
threshold = 0.01
s_t = 75
l_g = 1000
s_w = 100
l = 0
if strand == '+':
    l = l_g
n_w = (l_g - s_w)/s_t + 1
N_w = N * s_w / l_g 
m = poisson.ppf(threshold, N_w)
start_wi = 0

# Count reads for each window and check quantile condition
for i in range(int(n_w)):
    start_loc = start_wi + first_loc
    count = samfile.count(chrom, start=start_loc, end=start_loc+s_w)
    if count <= m:
        l = start_wi
    start_wi += s_t
l

In [ ]:
# Finding introns
samfile.find_introns((read for read in samfile.fetch(chrom, start=3000000, end=5500000)))

### New Approach
Union all exons start window from end cut at 1000 bp from end

In [11]:
utr = pd.read_csv('mm10_3utr.bed', delimiter='\t',
                 names=['CHROM', 'LOC1', 'LOC2', 'EXON', 'STRAND'])
bed = pd.read_csv('mm10_final.bed', delimiter='\t', 
                  names=['CHROM', 'LOC1', 'LOC2', 'NAME', 'STRAND'],
                  converters={'CHROM': str.strip, 'NAME': str.strip, 'STRAND': str.strip})

In [12]:
bed

,CHROM,LOC1,LOC2,NAME,STRAND
0,1,3214481,3671498,Xkr4,-
1,1,3648310,3658904,AK149000,-
2,1,4343506,4360314,Rp1,-
3,1,4490927,4497354,Sox17,-
4,1,4773199,4785726,Mrpl15,-
5,1,4807892,4846735,Lypla1,+
6,1,4857693,4897909,Tcea1,+
7,1,4909575,5070285,Rgs20,-
8,1,5083085,5162549,Atp6v1h,+
9,1,5588492,5606133,Oprk1,+


In [ ]:
samfile = pysam.AlignmentFile("P2_E01_529_mono_01.sorted.bam", "rb")
chrom = '10'
# next_loc = 4845673
# next_loc = 5149991
# next_loc = 80796115
threshold = 0.01
s_t = 75
s_w = 100
l_g = 1000
windows = []
# First find the gene associated to the beginning of the new transcript, then select all exons for that gene
loc = 16688488
gene = bed[(bed['CHROM'] == chrom) & (bed['LOC1'] <= loc) & (bed['LOC2'] >= loc)]
gene_name = gene['NAME'].iloc[0]
gene_exons = utr[utr['EXON'].str.contains(gene_name)]

In [ ]:
# Case strand == '+', window list
l = 0
first_loc = 164080785
chrom = '1'
strand = '+'
end = gene_exons['LOC2'].iloc[-1]
start = end - 1000
n_w = (l_g - s_w)/s_t + 1
N_w = N * s_w / l_g 
m = poisson.ppf(threshold, N_w)
start_wi = end
for i in range(int(n_w)):
    count = samfile.count('1', start=start_wi-s_w, end=start_wi)
    windows.append(count)
    start_wi -= s_t
windows

In [ ]:
# Case strand == '+'
l = 0
end = gene_exons['LOC2'].iloc[-1]
start = end - 1000
N = samfile.count(chrom, start=start, end=end)
n_w = (l_g - s_w)/s_t + 1
N_w = N * s_w / l_g 
m = poisson.ppf(threshold, N_w)
loc_wi = end
start_wi = 0
prev_mid = l + (s_w/2)
for i in range(int(n_w)):
    count = samfile.count('1', start=loc_wi-s_w, end=loc_wi)
    print('Count: {}'.format(count))
    if count <= m:
        prev_mid = start_wi + (s_w/2)
        start_wi += s_t
        print('Moving start to: {}'.format(start_wi))
    else:
        new_mid = start_wi + (s_w/2)
        l = int(np.ceil((prev_mid + new_mid) / 2))
        print('Prev: {}, New: {}, l = {}'.format(prev_mid, new_mid, l))
        break
    loc_wi -= s_t
l

In [2]:
# All the functions
def locate_termination(samfile, chrom, strand, gene_exons):
    
    # Initialize params
    threshold = 0.01
    s_t = 75
    s_w = 100
    l_g = 1000
    n_w = (l_g - s_w)/s_t + 1
    
    if strand == '-':
        start = gene_exons['LOC1'].iloc[0]
        end = start + 1000
        N = samfile.count(chrom, start=start, end=end)
        N_w = N * s_w / l_g 
        m = poisson.ppf(threshold, N_w)

    else:
        end = gene_exons['LOC2'].iloc[-1]
        start = end - 1000
        N = samfile.count(chrom, start=start, end=end)
        N_w = N * s_w / l_g 
        m = poisson.ppf(threshold, N_w)

    return slide_window(samfile, chrom, start, end, strand, n_w, m, s_w, s_t)

def slide_window(samfile, chrom, start, end, strand, n_w, m, s_w, s_t):
    
    # Init params
    l = 0
    start_wi = 0
    
    # Move sliding window according to strand
    if strand == '-':
        loc_wi = start
        prev_mid = l + (s_w/2)
        
        for i in range(int(n_w)):
            count = samfile.count(chrom, start=loc_wi, end=loc_wi+s_w)
            if count <= m:
                prev_mid = start_wi + (s_w/2)
                start_wi += s_t
            else:
                new_mid = start_wi + (s_w/2)
                l = int(np.ceil((prev_mid + new_mid) / 2))
                return l
            loc_wi += s_t
    else:
        loc_wi = end
        prev_mid = l + (s_w/2)
        
        for i in range(int(n_w)):
            count = samfile.count(chrom, start=loc_wi-s_w, end=loc_wi)
            if count <= m:
                prev_mid = start_wi + (s_w/2)
                start_wi += s_t
            else:
                new_mid = start_wi + (s_w/2)
                l = int(np.ceil((prev_mid + new_mid) / 2))
                return l
            loc_wi -= s_t
            
    return l

def get_gene_info(bed, utr, chrom, next_loc):
    try:
        gene = bed[(bed['CHROM'] == chrom) & (bed['LOC1'] <= next_loc) & (bed['LOC2'] >= next_loc)]
        gene_name = gene['NAME'].iloc[0]
        return gene_name, utr[utr['EXON'].str.contains(gene_name)]
    except IndexError:
        return None, None
    
def get_gene_end(bed, loc, chrom):
    gene = bed[(bed['CHROM'] == chrom) & (bed['LOC1'] <= loc) & (bed['LOC2'] >= loc)]
    return gene['LOC2'].iloc[0]

In [ ]:
samfile = pysam.AlignmentFile("P2_E01_529_mono_01.sorted.bam", "rb")
# next_loc = 4845673
# next_loc = 5149991
# next_loc = 80796115
next_locs = [4845673, 5149991]
chrom = '1'
strand = '+'
for next_loc in next_locs:
    gene_exons = get_gene_exons(bed, utr, chrom, next_loc)
    print(locate_termination(samfile, chrom, strand, gene_exons))

In [ ]:
%%time
# Working algorithm

# Load samfile
samfile = pysam.AlignmentFile("P2_E01_529_mono_01.sorted.bam", "rb")

# Load mm10 bed files
utr = pd.read_csv('mm10_3utr.bed', delimiter='\t')
bed = pd.read_csv('mm10_final.bed', delimiter='\t', converters={'CHROM': str.strip, 'NAME': str.strip, 'STRAND': str.strip})

chromosomes = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17', '18', '19', 'X', 'Y']
# chromosomes = ['1']
term_bed = bed
term_bed['TERM'] = 0
for chrom in chromosomes:
    print('CHROM: {}'.format(chrom))
    border = 0
    for read in samfile.fetch(chrom):
        start_position = read.get_reference_positions()[0]
        if start_position <= border: # Skip read if in region already accounted for
            continue
        gene_name, gene_exons = get_gene_info(bed, utr, chrom, start_position)
        if gene_exons is not None:
            print('Working start: {} passed'.format(start_position))
            strand = gene_exons['STRAND'].iloc[0]
            term_location = locate_termination(samfile, chrom, strand, gene_exons)
            term_bed.loc[term_bed['NAME'] == gene_name, 'TERM'] = term_location
            border = get_gene_end(bed, start_position)
            print('Strand: {} Loc: {} New Border: {}'.format(strand, term_location, border))
        else:
            continue

term_bed.to_csv('termination.csv', sep='\t')

In [13]:
# Working algorithm - function version

def termination(samfile):

    # Load mm10 bed files
    utr = pd.read_csv('mm10_3utr.bed', delimiter='\t')
    bed = pd.read_csv('mm10_final.bed', delimiter='\t', converters={'CHROM': str.strip, 'NAME': str.strip, 'STRAND': str.strip})

    chromosomes = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17', '18', '19', 'X', 'Y']
#     chromosomes = ['1']
    term_bed = bed
    term_bed['TERM'] = 0
    for chrom in chromosomes:
        border = 0
        for read in samfile.fetch(chrom):
            start_position = read.get_reference_positions()[0]
            if start_position <= border: # Skip read if in region already accounted for
                continue
            gene_name, gene_exons = get_gene_info(bed, utr, chrom, start_position)
            if gene_exons is not None:
                strand = gene_exons['STRAND'].iloc[0]
                term_location = locate_termination(samfile, chrom, strand, gene_exons)
                term_bed.loc[term_bed['NAME'] == gene_name, 'TERM'] = term_location
                border = get_gene_end(bed, start_position, chrom)
            else:
                continue
    return term_bed

In [12]:
# Load samfiles
samfile = pysam.AlignmentFile("P2_E01_529_mono_01.sorted.bam", "rb")

ter = termination(samfile)
ter

,CHROM,LOC1,LOC2,NAME,STRAND,TERM
0,1,3214481,3671498,Xkr4,-,0
1,1,3648310,3658904,AK149000,-,0
2,1,4343506,4360314,Rp1,-,0
3,1,4490927,4497354,Sox17,-,0
4,1,4773199,4785726,Mrpl15,-,0
5,1,4807892,4846735,Lypla1,+,50
6,1,4857693,4897909,Tcea1,+,0
7,1,4909575,5070285,Rgs20,-,0
8,1,5083085,5162549,Atp6v1h,+,88
9,1,5588492,5606133,Oprk1,+,0


### Testing parallelism

In [48]:
def termination_with_names(bam_name):
    
    # Load bam
    samfile = pysam.AlignmentFile(bam_name, "rb")

    # Load mm10 bed files
    utr = pd.read_csv('mm10_3utr.bed', delimiter='\t')
    bed = pd.read_csv('mm10_final.bed', delimiter='\t', converters={'CHROM': str.strip, 'NAME': str.strip, 'STRAND': str.strip})

    chromosomes = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17', '18', '19', 'X', 'Y']
#     chromosomes = ['1']
    term_bed = bed
    term_bed['TERM'] = 0
    for chrom in chromosomes:
        border = 0
        for read in samfile.fetch(chrom):
            start_position = read.get_reference_positions()[0]
            if start_position <= border: # Skip read if in region already accounted for
                continue
            gene_name, gene_exons = get_gene_info(bed, utr, chrom, start_position)
            if gene_exons is not None and not gene_exons.empty:
#                 print('Chrom: {} Name: {} Pos: {}'.format(chrom, gene_name, start_position))
                strand = gene_exons['STRAND'].iloc[0]
                term_location = locate_termination(samfile, chrom, strand, gene_exons)
                term_bed.loc[term_bed['NAME'] == gene_name, 'TERM'] = term_location
                border = get_gene_end(bed, start_position, chrom)
#                 print('Term_location: {} Border: {}'.format(term_location, border))
            else:
                continue
    print('Done with {}'.format(bam_name))
    return term_bed

In [51]:
%%time

# Testing normal procedure

bam_1 = "P2_E01_529_mono_01.sorted.bam"
bam_2 = "testing-parallelism/P2_E01_529_mono_02.sorted.bam"
bam_3 = "testing-parallelism/P2_E01_529_mono_03.sorted.bam"
bam_4 = "testing-parallelism/P2_E01_529_mono_04.sorted.bam"
bam_5 = "testing-parallelism/P2_E01_529_mono_05.sorted.bam"

term_1 = termination_with_names(bam_1)
term_2 = termination_with_names(bam_2)
term_3 = termination_with_names(bam_3)
term_4 = termination_with_names(bam_4)
term_5 = termination_with_names(bam_5)


Done with P2_E01_529_mono_01.sorted.bam


/home/lemac/miniconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """


Done with testing-parallelism/P2_E01_529_mono_02.sorted.bam
Done with testing-parallelism/P2_E01_529_mono_03.sorted.bam
Done with testing-parallelism/P2_E01_529_mono_04.sorted.bam
Done with testing-parallelism/P2_E01_529_mono_05.sorted.bam
CPU times: user 43min 55s, sys: 10.1 s, total: 44min 5s
Wall time: 44min 6s


In [52]:
%%time 

# Testing parallelism

# bam_1 = pysam.AlignmentFile("P2_E01_529_mono_01.sorted.bam", "rb")
# bam_2 = pysam.AlignmentFile("testing-parallelism/P2_E01_529_mono_02.sorted.bam", "rb")
# bam_3 = pysam.AlignmentFile("testing-parallelism/P2_E01_529_mono_03.sorted.bam", "rb")
# bam_4 = pysam.AlignmentFile("testing-parallelism/P2_E01_529_mono_04.sorted.bam", "rb")

bam_1 = "P2_E01_529_mono_01.sorted.bam"
bam_2 = "testing-parallelism/P2_E01_529_mono_02.sorted.bam"
bam_3 = "testing-parallelism/P2_E01_529_mono_03.sorted.bam"
bam_4 = "testing-parallelism/P2_E01_529_mono_04.sorted.bam"
bam_5 = "testing-parallelism/P2_E01_529_mono_05.sorted.bam"


bams = [bam_1, bam_2, bam_3, bam_4, bam_5]

num_cores = multiprocessing.cpu_count()

results = Parallel(n_jobs=num_cores)(delayed(termination_with_names)(bam) for bam in bams)

/home/lemac/miniconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """
/home/lemac/miniconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """


Done with testing-parallelism/P2_E01_529_mono_02.sorted.bam
Done with testing-parallelism/P2_E01_529_mono_03.sorted.bam
Done with testing-parallelism/P2_E01_529_mono_04.sorted.bam
Done with P2_E01_529_mono_01.sorted.bam
Done with testing-parallelism/P2_E01_529_mono_05.sorted.bam
CPU times: user 1.35 s, sys: 325 ms, total: 1.67 s
Wall time: 27min 47s


In [10]:
%%time 

# Testing parallelism with numba
from joblib import Parallel, delayed
import multiprocessing
from numba import jit
@jit
def termination_with_names(bed_name, utr_name, bam_name):
    
    # Load bam
    samfile = pysam.AlignmentFile(bam_name, "rb")

    # Load mm10 bed files
    utr = pd.read_csv(utr_name, delimiter='\t')
    bed = pd.read_csv(bed_name, delimiter='\t', converters={'CHROM': str.strip, 'NAME': str.strip, 'STRAND': str.strip})

    chromosomes = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17', '18', '19', 'X', 'Y']
#     chromosomes = ['1']
    term_bed = bed
    term_bed['TERM'] = 0
    for chrom in chromosomes:
        border = 0
        for read in samfile.fetch(chrom):
            start_position = read.get_reference_positions()[0]
            if start_position <= border: # Skip read if in region already accounted for
                continue
            gene_name, gene_exons = get_gene_info(bed, utr, chrom, start_position)
            if gene_exons is not None and not gene_exons.empty:
#                 print('Chrom: {} Name: {} Pos: {}'.format(chrom, gene_name, start_position))
                strand = gene_exons['STRAND'].iloc[0]
                term_location = locate_termination(samfile, chrom, strand, gene_exons)
                term_bed.loc[term_bed['NAME'] == gene_name, 'TERM'] = term_location
                border = get_gene_end(bed, start_position, chrom)
#                 print('Term_location: {} Border: {}'.format(term_location, border))
            else:
                continue
    print('Done with {}'.format(bam_name))
    return term_bed

# bam_1 = pysam.AlignmentFile("P2_E01_529_mono_01.sorted.bam", "rb")
# bam_2 = pysam.AlignmentFile("testing-parallelism/P2_E01_529_mono_02.sorted.bam", "rb")
# bam_3 = pysam.AlignmentFile("testing-parallelism/P2_E01_529_mono_03.sorted.bam", "rb")
# bam_4 = pysam.AlignmentFile("testing-parallelism/P2_E01_529_mono_04.sorted.bam", "rb")

utr_name = 'mm10_3utr.bed'
bed_name = 'mm10_final.bed'
    
bam_1 = "P2_E01_529_mono_01.sorted.bam"
bam_2 = "testing-parallelism/P2_E01_529_mono_02.sorted.bam"
bam_3 = "testing-parallelism/P2_E01_529_mono_03.sorted.bam"
bam_4 = "testing-parallelism/P2_E01_529_mono_04.sorted.bam"
bam_5 = "testing-parallelism/P2_E01_529_mono_05.sorted.bam"


bams = [bam_1, bam_2]

num_cores = multiprocessing.cpu_count()

results2 = Parallel(n_jobs=num_cores)(delayed(termination_with_names)(bed_name, utr_name, bam) for bam in bams)

Process ForkPoolWorker-7:
Traceback (most recent call last):
  File "/home/lemac/miniconda3/envs/ml/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/lemac/miniconda3/envs/ml/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/lemac/miniconda3/envs/ml/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/lemac/miniconda3/envs/ml/lib/python3.6/site-packages/joblib/pool.py", line 362, in get
    return recv()
  File "/home/lemac/miniconda3/envs/ml/lib/python3.6/multiprocessing/connection.py", line 250, in recv
    buf = self._recv_bytes()
  File "/home/lemac/miniconda3/envs/ml/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/home/lemac/miniconda3/envs/ml/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt


KeyboardInterrupt: 

In [39]:
def create_txt_files(term_list):
    df = term_list[0][['NAME', 'TERM']]
    for i in range(1, len(term_list)):
        df[str(i)] = term_list[i][['TERM']]
    name = 'five'
        
    df.to_csv(name + 'df.txt', header=None, index=None, sep='\t', mode='a')

In [40]:
create_txt_files(results2)

/home/lemac/miniconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
